# Convolutional Neural Network (CNN) Model for Image Classification of Smog vs Clear images

Import all libraries and set environment 

In [ ]:
import numpy as np
from random import shuffle
import cv2
from tqdm import tqdm
import os

install TFlearn from Tensorflow

In [ ]:
!pip install tflearn

In [ ]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression


In [ ]:
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive 

For collaboration, imported data from google drive || Run also using locally using Jupyter Notebooks

In [ ]:
drive.mount('/content/drive')

Assignening data into variables

In [ ]:
# Training and Testing data 

entrenamiento = #Path training data || Drive
dtest = #Path Testing Data || Drive

LR = 1e-3
tamano= 50 ## Figure size || Subject to change depending on data pre-processing 

### Creating the model as a basic CNN formated 1e -3

In [ ]:
#Model
modelo = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic')

### Labelling the data (Smog vs clear)

In [ ]:
#Def to Label
def label_img(img):
	word_label = img.split('.')[-3]
	if word_label == 'smog': return [1, 0]
	elif word_label == 'clear': return [0, 1]

### Creating Dataset only for training data 
Missing a couple shape details / for image size 

In [ ]:
def ctd(): 
	training_data = []
	for img in tqdm(os.listdir(entrenamiento)):
		label = label_img(img)
		path = os.path.join(entrenamiento, img)
		img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
		#reshpae 
		img = cv2.resize(img, (tamano, tamano))
		training_data.append([np.array(img), np.array(label)])
	# Preservar random 
	shuffle(training_data)
	np.save('train_data.npy', training_data)
	return training_data

### Procesing test data

In [ ]:
def ptd(): 
	testing_data = []
	for img in tqdm(os.listdir(dtest)):
		path = os.path.join(dtest, img)
		img_num = img.split('.')[0]
		img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
		img = cv2.resize(img, (tamano, tamano))
		testing_data.append([np.array(img), img_num])
		
	shuffle(testing_data)
	np.save('test_data.npy', testing_data)
	return testing_data

Assign data to train and test

In [ ]:
train_data = ctd()
test_data = ptd()

## Creat CNN : 2d 

In [ ]:
tf.reset_default_graph()
#Layer 1
convnet = input_data(shape =[None, tamano, tamano, 1], name ='input')
convnet = conv_2d(convnet, 32, 5, activation ='relu')
#Layer 2
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation ='relu')
#Layer 3
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation ='relu')
#Layer 4
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation ='relu')
#Layer 5
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation ='relu')

convnet = max_pool_2d(convnet, 5)
#Fully connected CNN 
convnet = fully_connected(convnet, 1024, activation ='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 2, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = LR,
	loss ='categorical_crossentropy', name ='targets')
model = tflearn.DNN(convnet, tensorboard_dir ='log')

Store data to start training with same shape each 

In [ ]:
train = train_data[:-500]
test = train_data[-500:]

Fianl split of data, ready to fit 

In [ ]:
X = np.array([i[0] for i in train]).reshape(-1, tamano, tamano, 1)
Y = [i[1] for i in train]
test_x = np.array([i[0] for i in test]).reshape(-1, tamano, tamano, 1)
test_y = [i[1] for i in test]

### Fit model using 5 epochs

In [ ]:
#Datos en el modelo con 5 epochs 
model.fit({'input': X}, {'targets': Y}, n_epoch = 5,
	validation_set =({'input': test_x}, {'targets': test_y}),
	snapshot_step = 500, show_metric = True, run_id = modelo)
model.save(modelo)